In [ ]:
# Community Parking Data Scraping

In [ ]:
import requests 

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url="https://philapark.org/lots/"

In [ ]:
r=requests.get(url)

In [ ]:
# Parse and Extract Required Content Using Beautiful Soup

In [ ]:
soup=BeautifulSoup(r.text,'html.parser')

In [ ]:
soup.find('div',class_="tw-element span4").contents

In [ ]:
for j in soup.find('div',class_="tw-element span4").contents:
    if j.name=='div':
        print(j.text)
    if j.name=='h2':
        print(j.text)
    if j.name=='p':
        if j.find('strong')!=None:
            txt=' '.join([i.text for i in j.find_all('strong')])
            print(txt,)
            print(j.text[len(txt):])
        else:
            print(j.text)
        print("--------------")

In [ ]:
for i in soup.find_all('div',class_="tw-element span4"):
    for j in i.contents:
        if j.name=='div':
            print(j.text)
        if j.name=='h2':
            print(j.text)
        if j.name=='p':
            if j.find('strong')!=None:
                txt=' '.join([i.text for i in j.find_all('strong') if i.text!='(Currently Closed)'])
                print(txt,)
                print(j.text[len(txt):].rstrip('(Currently Closed)'))
            
            print()
    print('--------------')

In [ ]:
add=[]
for i in soup.find_all('div',class_="tw-element span4"):
    for j in i.contents:
        if j.name=='div':
            region=j.text
        if j.name=='h2':
            area=j.text
        if j.name=='p':
            if j.find('strong')!=None:
                txt=' '.join([i.text for i in j.find_all('strong') if i.text!='(Currently Closed)'])
                street=txt
                address=j.text[len(txt):].rstrip('(Currently Closed)')
                add.append([region,area,street,address])


In [ ]:
add[0]

In [ ]:
import csv

In [ ]:
# Save the Required COntent

In [ ]:
with open('parking_lot_data.csv','w',newline='') as f:
    cv=csv.writer(f, delimiter=',')
    cv.writerow(['Region','Area','Name','Address'])

    for i in range(len(add)):
        cv.writerow(add[i])

In [ ]:
# Existing Food Distrubution SItes Data Scraping Using Selenium and Beautiful Soup

In [7]:
from selenium import webdriver
path="geckodriver"

In [8]:
driver = webdriver.Firefox(executable_path=path)

In [ ]:
url="https://www.phila.gov/food/?services=General%20meal%20site"

In [ ]:
driver.get(url)

In [ ]:
addr=[]

In [ ]:
for i in driver.find_elements_by_class_name("location-item"):
    i.click()
    soup=BeautifulSoup(i.get_attribute('innerHTML'),'html.parser')
#     print(soup.find("h2",{'id':"bethel-presbyterian"}).text)
    addr.append(soup.find("h2").text+" - "+soup.find('div',{"class":"detail"}).text)

In [ ]:
addr

In [ ]:
len(addr)

In [ ]:
import csv

In [ ]:
with open('general_meal_site.csv','w',newline='') as f:
    cv=csv.writer(f, delimiter=',')
    cv.writerow(['Address'])

    for i in range(len(addr)):
        cv.writerow([addr[i]])

In [ ]:
# Scraping Data From Private Org Dataset Using Selenium and Beautiful Soup

In [ ]:
from selenium import webdriver
path="geckodriver"
geoDisabled = webdriver.FirefoxOptions()
geoDisabled.set_preference("geo.enabled", False)
driver = webdriver.Firefox(executable_path=path,options=geoDisabled)


In [ ]:
url="https://www.philabundance.org/find-food/food-map/"

In [ ]:
driver.get(url)

In [ ]:
driver.switch_to_alert().accept()

In [ ]:
source=driver.page_source

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup=BeautifulSoup(source)

In [ ]:
addr=" ".join(soup.find("div",{"class":"wpsl-store-location"}).text.split(" – ")[1].split("\n")[:-2])

In [ ]:
driver.find_element_by_id("wpsl-search-input").send_keys(addr)

In [ ]:
from selenium.webdriver.support.ui import Select

select = Select(driver.find_element_by_id('wpsl-radius-dropdown'))


In [ ]:
from selenium.webdriver.support.ui import Select

In [ ]:
driver.find_element_by_class_name("wpsl-dropdown").click()

In [ ]:
driver.find_element_by_xpath('//li[@data-value="10"]').click()

In [ ]:
driver.find_element_by_id("wpsl-results").find_element_by_class_name("wpsl-dropdown").click()

In [ ]:
driver.find_element_by_xpath('//li[@data-value="100"]').click()

In [ ]:
driver.find_element_by_id('wpsl-search-btn').click()

In [ ]:
html=driver.page_source

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup=BeautifulSoup(html,'html.parser')

In [ ]:
soup.find("span",{"class":"wpsl-street"})

In [ ]:
li=[]

In [ ]:
soup.find("div",{"class":"wpsl-store-location"}).find('p',{'class':'title'}).text.strip(' \n\t')

In [ ]:
for i in soup.find_all("div",{"class":"wpsl-store-location"}):
    name=i.find('p',{'class':'title'}).text.strip(' \n\t')
    add=""
    for j in i.find_all("span"):
        add+=j.text
        add+=" "
    add.strip(" ")
    li.append([name,add])


In [ ]:
li

In [ ]:
len(li)

In [ ]:
import csv

In [ ]:
with open('philabundance_updated.csv','w',newline='') as f:
    cv=csv.writer(f, delimiter=',')
    cv.writerow(['Name','Address'])

    for i in range(len(li)):
        if li[i][0]==li[i-1][0]:
            continue
        cv.writerow(li[i])

In [ ]:
# Free Library Wifi Locations Scrapping

In [ ]:
import requests as rq

In [ ]:
url="https://libwww.freelibrary.org/locations/service/wifi-access"

In [ ]:
r=rq.get(url)

In [ ]:
r.status_code

In [ ]:
soup=BeautifulSoup(r.text,'html.parser')

In [ ]:
addr=[]

In [ ]:
for i in soup.find_all("div",{'class':'col-sm-4'}):
    a=""
    c=0
    if 'Currently Closed' in i.find('div').text.strip(' \r\n') or 'By Appointment' in i.find('div').text.strip(' \r\n\xa0'):
        continue
    for j in i.find_all('div')[:3]:
        if c==0:
            a+=(j.text.strip('\n \r')+' - ')
            c+=1
        else:
            a+=(j.text.strip('\n \r')+' ')
            c+=1
    addr.append(a)


In [ ]:
addr

In [ ]:
len(addr)

In [ ]:

with open('wifi_locations.csv','w',newline='') as f:
    cv=csv.writer(f, delimiter=',')
    cv.writerow(['Address'])

    for i in range(len(addr)):
        cv.writerow([addr[i]])

In [ ]:
# Free Wifi Data Scrapping

In [ ]:
import requests

In [ ]:
url="https://phl.maps.arcgis.com/apps/webappviewer/index.html?id=928cdaef12994bd1b3968da825a71a1a"

In [ ]:
r=requests.get(url)

In [ ]:
r.status_code

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup=BeautifulSoup(r.text)

In [ ]:
soup.find('table')

In [10]:
from selenium import webdriver
path="geckodriver"

In [ ]:
driver = webdriver.Firefox(executable_path=path)

In [ ]:
driver.get(url)

In [ ]:
driver.page_source

In [ ]:
soup=BeautifulSoup(driver.page_source)

In [ ]:
td_728=soup.find('div',{"class":"dgrid-scroller","tabindex":"-1"}).find_all('tr')

In [ ]:
td_728[0]

In [ ]:
td_728[-1]

In [ ]:
soup=BeautifulSoup(driver.page_source)

In [ ]:
td_612=soup.find('div',{"class":"dgrid-scroller","tabindex":"-1"}).find_all('tr')

In [ ]:
td_612[0]

In [ ]:
td_612[-1]

In [ ]:
table_612=[]

In [ ]:
for i in td_612:
    table_612.append([j.text for j in i.find_all('td')[1:]])
#     print([j.text for j in i.find_all('td')[1:]])

In [ ]:
len(table_612)

In [ ]:
for i in table_612:
    print(i)

In [ ]:
td_728[35]

In [ ]:
table_728=[]

In [ ]:
for i in td_728[35:]:
    table_728.append([j.text for j in i.find_all('td')[1:]])

In [ ]:
table_728[-1]

In [ ]:
len(table_612+table_728)

In [ ]:
free_wifi=table_612+table_728

In [ ]:
import csv

In [ ]:
with open('free_wifi.csv','w',newline='') as f:
    cv=csv.writer(f, delimiter=',')
    cv.writerow(['ObjectID',"Street Address","Hotspot Type","Name"])

    for i in range(len(free_wifi)):
        cv.writerow(free_wifi[i])

In [ ]:
# Getting Distance b/w two points through address using Google API Key 
# and Googlemaps Library
# Hiding google api key as it is a paid version

In [2]:
key="Your_AP_Key"

In [1]:
import googlemaps

In [36]:
gmaps = googlemaps.Client(key=key)
  
source="55 North Broad Street Philadelphia PA 19107"
destination="123 South 17th Street Philadelphia PA 19103"
my_dist = gmaps.distance_matrix(source,destination)['rows'][0]['elements'][0]
  

print(my_dist)

{'distance': {'text': '2.0 km', 'value': 2015}, 'duration': {'text': '9 mins', 'value': 511}, 'status': 'OK'}


In [1]:
# Getting Latitudes and Longitudes of a address

In [70]:
# key="Your_API_Key" #Google API Key #Cannot insert API key in public domain as its a paid version.

In [71]:
import requests

In [ ]:
# Getting Lattitude and Longitude of an address using google maps API

In [72]:
url="https://maps.googleapis.com/maps/api/geocode/json?"

In [73]:
add="1900 W York St Philadelphia, PA 19132"

In [77]:
add="2200 Turner Street Philadelphia, PA 19121"

In [78]:
params={'key':key,
       'address':add}

In [79]:
r=requests.get(url,params=params)


In [81]:
r.json()

{'results': [{'address_components': [{'long_name': '2200',
     'short_name': '2200',
     'types': ['street_number']},
    {'long_name': 'Turner Street',
     'short_name': 'Turner St',
     'types': ['route']},
    {'long_name': 'North Philadelphia',
     'short_name': 'North Philadelphia',
     'types': ['neighborhood', 'political']},
    {'long_name': 'Philadelphia',
     'short_name': 'Philadelphia',
     'types': ['locality', 'political']},
    {'long_name': 'Philadelphia County',
     'short_name': 'Philadelphia County',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'Pennsylvania',
     'short_name': 'PA',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United States',
     'short_name': 'US',
     'types': ['country', 'political']},
    {'long_name': '19121', 'short_name': '19121', 'types': ['postal_code']}],
   'formatted_address': '2200 Turner St, Philadelphia, PA 19121, USA',
   'geometry': {'location': {'lat':

In [80]:
r.json()['results'][0]['geometry']['location']

{'lat': 39.9792289, 'lng': -75.1712891}

In [ ]:
# Updating Datasets

In [58]:
import pandas as pd

In [59]:
df=pd.read_csv('wifi_locations.csv')

In [63]:
df['Name']=df['Address'].apply(lambda a:a.split(' - ')[0])

In [65]:
df['Address']=df['Address'].apply(lambda a:a.split(' - ')[1])

In [67]:
df=df[['Name','Address']]

In [68]:
df

,Name,Address
0,Andorra Library,"705 East Cathedral Road Philadelphia, PA 19128..."
1,Blanche A. Nixon/Cobbs Creek Library,"5800 Cobbs Creek Parkway Philadelphia, PA 1914..."
2,Bushrod Library,"6304 Castor Avenue Philadelphia, PA 19149-2731"
3,Bustleton Library,"10199 Bustleton Avenue Philadelphia, PA 19116-..."
4,Cecil B. Moore Library,"2320 Cecil B. Moore Avenue Philadelphia, PA 19..."
5,Charles L. Durham Library,"3320 Haverford Avenue Philadelphia, PA 19104-2..."
6,Charles Santore Library,"932 South 7th Street Philadelphia, PA 19147-2932"
7,Chestnut Hill Library,"8711 Germantown Avenue Philadelphia, PA 19118-..."
8,David Cohen Ogontz Library,"6017 Ogontz Avenue Philadelphia, PA 19141"
9,Eastwick Library,"2851 Island Avenue Philadelphia, PA 19153-2314"


In [70]:
df.to_csv('wifi_locations_updated.csv',index=False)

In [79]:
with open('free_wifi.csv','r') as f:
    data=f.read()

In [80]:
data

'ObjectID,Street Address,Hotspot Type,Name\n143431,2000 Market Street,LinkPHL Kiosk,LinkPHL Kiosk\n143432,2104 Market Street,LinkPHL Kiosk,LinkPHL Kiosk\n143433,1835 Market St,LinkPHL Kiosk,LinkPHL Kiosk\n143434,1515 Market St,LinkPHL Kiosk,LinkPHL Kiosk\n143435,1700 Market St,LinkPHL Kiosk,LinkPHL Kiosk\n143436,51 N 15Th St,LinkPHL Kiosk,LinkPHL Kiosk\n143437,1 S Penn Square,LinkPHL Kiosk,LinkPHL Kiosk\n143438,1880 John F Kennedy Blvd,LinkPHL Kiosk,LinkPHL Kiosk\n143439,1701 Arch St,LinkPHL Kiosk,LinkPHL Kiosk\n143440,1724 Arch St,LinkPHL Kiosk,LinkPHL Kiosk\n143441,99 N 20Th St,LinkPHL Kiosk,LinkPHL Kiosk\n143442,1650 Market St,LinkPHL Kiosk,LinkPHL Kiosk\n143443,2130 Arch St,LinkPHL Kiosk,LinkPHL Kiosk\n143444,1515 John F Kennedy Blvd,LinkPHL Kiosk,LinkPHL Kiosk\n143445,5704 N 5Th Street,LinkPHL Kiosk,LinkPHL Kiosk\n143446,2829 W. Diamond St,KEYSPOT,Strawberry Mansion Neighborhood Action Center \n143447,1936 N. Judson St,KEYSPOT,Project HOME - Honickman Learning Center\n143449,North

In [81]:
df=pd.read_csv('free_wifi.csv')

In [86]:
df

,ObjectID,Street Address,Hotspot Type,Name
0,143431,2000 Market Street Philadelphia,LinkPHL Kiosk,LinkPHL Kiosk
1,143432,2104 Market Street Philadelphia,LinkPHL Kiosk,LinkPHL Kiosk
2,143433,1835 Market St Philadelphia,LinkPHL Kiosk,LinkPHL Kiosk
3,143434,1515 Market St Philadelphia,LinkPHL Kiosk,LinkPHL Kiosk
4,143435,1700 Market St Philadelphia,LinkPHL Kiosk,LinkPHL Kiosk
...,...,...,...,...
264,143723,10153-99 Bustleton Ave Philadelphia,Free Library,Library Branch - Bustleton Avenue
265,143724,501 Rhawn St Philadelphia,Free Library,Library Branch - Fox Chase
266,143725,11099 Knights Rd Philadelphia,Free Library,Library Branch - Katherine Drexel
267,143726,600-16 W Girard Ave Philadelphia,Free Library,Library Branch - Ramonita de Rodriguez


In [85]:
df['Street Address']=df['Street Address'].apply(lambda a:a+' Philadelphia')

In [88]:
df.to_csv('free_wifi_updated.csv',index=False)

In [ ]:
# Getting Distance b/w two points using Latitudes and Longitudes

In [3]:
from haversine import haversine,Unit

In [4]:
from geopy.distance import geodesic

In [5]:
# Loading the lat-long data for Kolkata & Delhi
kolkata = (22.5726, 88.3639)
delhi = (28.7041, 77.1025)
  
# Print the distance calculated in km
print(geodesic(kolkata, delhi).km)

1318.13891581683


In [6]:
print(haversine(kolkata,delhi,unit=Unit.KILOMETERS))

1317.7554231984702


In [7]:
# Creating Network Points

In [8]:
import pandas as pd
import numpy as np

In [47]:
df_parking=pd.read_csv('parking_lot_data.csv',encoding='cp1252')

In [48]:
df_parking.head()

,Region,Area,Name,Address,Lat,Long
0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289
1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734
2,North Philadelphia,Fairhill,West Somerset Street Lot,"401 W. Somerset StreetPhiladelphia, PA 19133",39.993999,-75.137940
3,North Philadelphia,Fairhill,5th & Indiana Street Lot,"5th Street & Indiana Street Philadelphia, PA 1...",40.003162,-75.137864
4,North Philadelphia,Strawberry Mansion,32nd & West Lehigh Avenue Lot,"3250 W. Lehigh AvenuePhiladelphia, PA 19132",39.997867,-75.184748


In [49]:
df_parking.iloc[10,:].name

10

In [50]:
df_parking['Id']=df_parking.apply(lambda a:('PL-'+str(a.name)),axis=1)

In [51]:
df_parking=df_parking[['Id','Region','Area','Name','Address','Lat','Long']]

In [52]:
df_parking

,Id,Region,Area,Name,Address,Lat,Long
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734
2,PL-2,North Philadelphia,Fairhill,West Somerset Street Lot,"401 W. Somerset StreetPhiladelphia, PA 19133",39.993999,-75.137940
3,PL-3,North Philadelphia,Fairhill,5th & Indiana Street Lot,"5th Street & Indiana Street Philadelphia, PA 1...",40.003162,-75.137864
4,PL-4,North Philadelphia,Strawberry Mansion,32nd & West Lehigh Avenue Lot,"3250 W. Lehigh AvenuePhiladelphia, PA 19132",39.997867,-75.184748
5,PL-5,North Philadelphia,West Kensington,East Huntingdon Street Lot,"301 E. Huntingdon StreetPhiladelphia, PA 19125",39.989288,-75.127644
6,PL-6,North Philadelphia,West Poplar,6th & Marshall Street Lot,"982 N. 6th Street Philadelphia, PA 19123",39.970208,-75.146787
7,PL-7,North Philadelphia,Logan,North Broad Street Lot,"4850 N. Broad StreetPhiladelphia, PA 19141",40.027449,-75.147563
8,PL-8,North Philadelphia,Logan,Fern Rock Rail Station Parking Lot,"901 Nedro AvenuePhiladelphia, PA 19141",40.042499,-75.135363
9,PL-9,North Philadelphia,Olney,Fairhill Street Lot,"5541 Fairhill StreetPhiladelphia, PA 19120",40.036382,-75.131405


In [53]:
df_parking.to_csv('updated_park_lot.csv',index=False)

In [54]:
df_gen_meal=pd.read_csv('general_meal_site_updated.csv',encoding='cp1252')

In [55]:
df_gen_meal['Id']=df_gen_meal.apply(lambda a:'GM-'+str(a.name),axis=1)

In [56]:
df_gen_meal

,Name,Address,Lat,Long,Id
0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.990390,-75.164210,GM-0
1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.946540,-75.164670,GM-1
2,Chosen 300 General meal site,"1116 Spring Garden St. Philadelphia, PA 19123",39.961570,-75.156300,GM-2
3,Chosen 300 West General meal site,"3959 Lancaster Ave Philadelphia, PA 19104",39.963050,-75.202010,GM-3
4,Church of the Advocate General meal site,"1801 Diamond St Philadelphia, PA 19121",39.985690,-75.163370,GM-4
5,Face to Face General meal site,"123 E. Price St Philadelphia, PA 19144",40.038220,-75.174020,GM-5
6,Grace Cafe General meal site,"55 N Broad St Philadelphia, PA 19107",39.953640,-75.163140,GM-6
7,Hub of Hope General meal site,"1401 John F Kennedy Blvd Philadelphia, PA 19102",39.953540,-75.163880,GM-7
8,Living Word Community General meal site,"142 N. 17th St Philadelphia, PA 19103",39.956460,-75.167800,GM-8
9,Mt. Tabor AME Church General meal site,"961 N. 7th St Philadelphia, PA 19123",39.967990,-75.148480,GM-9


In [57]:
df_gen_meal=df_gen_meal[['Id','Name','Address','Lat','Long']]

In [58]:
df_gen_meal.to_csv('updated_gen_meal_site.csv',index=False)

In [60]:
df_priv_meal=pd.read_csv('philabundance_updated.csv',encoding='cp1252')

In [62]:
df_priv_meal['Id']=df_priv_meal.apply(lambda a:'PM-'+str(a.name),axis=1)

In [65]:
df_priv_meal=df_priv_meal[['Id','Name','Address','Lat','Long']]

In [73]:
df_priv_meal.to_csv('updated_pilabundance.csv',index=False)

In [76]:
df_lib_loc=pd.read_csv('wifi_locations_updated.csv',encoding='cp1252')

In [77]:
df_lib_loc['Id']=df_lib_loc.apply(lambda a:'WL-'+str(a.name),axis=1)

In [80]:
df_lib_loc=df_lib_loc[['Id','Name','Address','Lat','Long']]

In [81]:
df_lib_loc.to_csv('updated_library_wifi.csv',index=False)

In [84]:
df_parking.head(2)

,Id,Region,Area,Name,Address,Lat,Long
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734


In [85]:
df_lib_loc.head(2)

,Id,Name,Address,Lat,Long
0,WL-0,Andorra Library,"705 East Cathedral Road Philadelphia, PA 19128...",40.06489,-75.23608
1,WL-1,Blanche A. Nixon/Cobbs Creek Library,"5800 Cobbs Creek Parkway Philadelphia, PA 1914...",39.94588,-75.24119


In [86]:
df_gen_meal.head(2)

,Id,Name,Address,Lat,Long
0,GM-0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.99039,-75.16421
1,GM-1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.94654,-75.16467


In [87]:
df_priv_meal.head(2)

,Id,Name,Address,Lat,Long
0,PM-0,Center – Philadelphia,55 North Broad Street Philadelphia PA 19107 Un...,39.95364,-75.16314
1,PM-1,Sunday Breakfast Rescue Mission,302 North 13th Street Philadelphia PA 19107 Un...,39.95834,-75.16022


In [ ]:
print(haversine(kolkata,delhi,unit=Unit.KILOMETERS))

In [317]:
df_network=df_parking

In [318]:
df_network['Wifi_Points']=0

In [319]:
df_network['Wifi_Locs']=''

In [320]:
df_network.head(2)

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,0,
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,0,


In [321]:
# df_network.iloc[0,-1]+=('2'+'_')

In [322]:
df_lib_loc.head(2)

,Id,Name,Address,Lat,Long
0,WL-0,Andorra Library,"705 East Cathedral Road Philadelphia, PA 19128...",40.06489,-75.23608
1,WL-1,Blanche A. Nixon/Cobbs Creek Library,"5800 Cobbs Creek Parkway Philadelphia, PA 1914...",39.94588,-75.24119


In [323]:
df_lib_loc.iloc[0,0]

'WL-0'

In [324]:
par_lat,par_lon=df_network.iloc[0,-4:-2].values
lib_lat,lib_lon=df_lib_loc.iloc[0,-2:].values
type(haversine((par_lat,par_lon),(lib_lat,lib_lon),unit=Unit.METERS))

TypeError: must be real number, not str

In [325]:
for i in range(len(df_network)):
    par_lat,par_lon=df_network.iloc[i,-4:-2].values
    for j in range(len(df_lib_loc)):
        lib_lat,lib_lon=df_lib_loc.iloc[j,-2:].values
        dis=haversine((par_lat,par_lon),(lib_lat,lib_lon),unit=Unit.METERS)
        if dis<=100:
            df_network.iloc[i,-2]+=1
            df_network.iloc[i,-1]+=(df_lib_loc.iloc[j,0]+'_')
        

TypeError: must be real number, not str

In [326]:
df_network

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,0,
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,0,
2,PL-2,North Philadelphia,Fairhill,West Somerset Street Lot,"401 W. Somerset StreetPhiladelphia, PA 19133",39.993999,-75.137940,0,,0,
3,PL-3,North Philadelphia,Fairhill,5th & Indiana Street Lot,"5th Street & Indiana Street Philadelphia, PA 1...",40.003162,-75.137864,0,,0,
4,PL-4,North Philadelphia,Strawberry Mansion,32nd & West Lehigh Avenue Lot,"3250 W. Lehigh AvenuePhiladelphia, PA 19132",39.997867,-75.184748,0,,0,
5,PL-5,North Philadelphia,West Kensington,East Huntingdon Street Lot,"301 E. Huntingdon StreetPhiladelphia, PA 19125",39.989288,-75.127644,0,,0,
6,PL-6,North Philadelphia,West Poplar,6th & Marshall Street Lot,"982 N. 6th Street Philadelphia, PA 19123",39.970208,-75.146787,0,,0,
7,PL-7,North Philadelphia,Logan,North Broad Street Lot,"4850 N. Broad StreetPhiladelphia, PA 19141",40.027449,-75.147563,0,,0,
8,PL-8,North Philadelphia,Logan,Fern Rock Rail Station Parking Lot,"901 Nedro AvenuePhiladelphia, PA 19141",40.042499,-75.135363,0,,0,
9,PL-9,North Philadelphia,Olney,Fairhill Street Lot,"5541 Fairhill StreetPhiladelphia, PA 19120",40.036382,-75.131405,0,,0,


In [327]:
df_network['Meal_Points']=0

In [328]:
df_network['Meal_Loc']=''

In [329]:
df_network.head(2)

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,0,
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,0,


In [330]:
df_gen_meal.head(2)

,Id,Name,Address,Lat,Long
0,GM-0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.99039,-75.16421
1,GM-1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.94654,-75.16467


In [331]:
df_priv_meal.head(2)

,Id,Name,Address,Lat,Long
0,PM-0,Center – Philadelphia,55 North Broad Street Philadelphia PA 19107 Un...,39.95364,-75.16314
1,PM-1,Sunday Breakfast Rescue Mission,302 North 13th Street Philadelphia PA 19107 Un...,39.95834,-75.16022


In [332]:
df_network.head(2)

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,0,
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,0,


In [333]:
df_meal=pd.concat([df_gen_meal,df_priv_meal],ignore_index=True)

In [334]:
df_meal.head(2)

,Id,Name,Address,Lat,Long
0,GM-0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.99039,-75.16421
1,GM-1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.94654,-75.16467


In [335]:
df_meal.iloc[0,-2:].values

array([39.99039, -75.16421], dtype=object)

In [336]:
df_network.iloc[0,-4:-2].values

array([0, ''], dtype=object)

In [337]:
for i in range(len(df_network)):
    par_lat,par_lon=df_network.iloc[i,-6:-4].values
    for j in range(len(df_meal)):
        meal_lat,meal_lon=df_meal.iloc[j,-2:].values
        dis=haversine((par_lat,par_lon),(meal_lat,meal_lon),unit=Unit.KILOMETERS)
        if dis<=2:
            df_network.iloc[i,-2]+=1
            df_network.iloc[i,-1]+=(df_meal.iloc[j,0]+'_')

In [338]:
df_network

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,13,GM-0_GM-4_PM-10_PM-16_PM-18_PM-25_PM-26_PM-32_...
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,25,GM-1_GM-2_GM-6_GM-7_GM-8_GM-9_GM-11_GM-14_GM-1...
2,PL-2,North Philadelphia,Fairhill,West Somerset Street Lot,"401 W. Somerset StreetPhiladelphia, PA 19133",39.993999,-75.137940,0,,11,GM-10_GM-13_PM-34_PM-35_PM-39_PM-41_PM-42_PM-4...
3,PL-3,North Philadelphia,Fairhill,5th & Indiana Street Lot,"5th Street & Indiana Street Philadelphia, PA 1...",40.003162,-75.137864,0,,9,GM-10_GM-13_PM-39_PM-41_PM-43_PM-45_PM-50_PM-5...
4,PL-4,North Philadelphia,Strawberry Mansion,32nd & West Lehigh Avenue Lot,"3250 W. Lehigh AvenuePhiladelphia, PA 19132",39.997867,-75.184748,0,,5,GM-0_PM-26_PM-38_PM-44_PM-49_
5,PL-5,North Philadelphia,West Kensington,East Huntingdon Street Lot,"301 E. Huntingdon StreetPhiladelphia, PA 19125",39.989288,-75.127644,0,,11,GM-10_GM-12_GM-13_PM-34_PM-35_PM-41_PM-42_PM-4...
6,PL-6,North Philadelphia,West Poplar,6th & Marshall Street Lot,"982 N. 6th Street Philadelphia, PA 19123",39.970208,-75.146787,0,,17,GM-2_GM-9_GM-11_GM-14_GM-16_GM-18_PM-1_PM-4_PM...
7,PL-7,North Philadelphia,Logan,North Broad Street Lot,"4850 N. Broad StreetPhiladelphia, PA 19141",40.027449,-75.147563,0,,0,
8,PL-8,North Philadelphia,Logan,Fern Rock Rail Station Parking Lot,"901 Nedro AvenuePhiladelphia, PA 19141",40.042499,-75.135363,0,,0,
9,PL-9,North Philadelphia,Olney,Fairhill Street Lot,"5541 Fairhill StreetPhiladelphia, PA 19120",40.036382,-75.131405,0,,0,


In [339]:
df_network

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,13,GM-0_GM-4_PM-10_PM-16_PM-18_PM-25_PM-26_PM-32_...
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,25,GM-1_GM-2_GM-6_GM-7_GM-8_GM-9_GM-11_GM-14_GM-1...
2,PL-2,North Philadelphia,Fairhill,West Somerset Street Lot,"401 W. Somerset StreetPhiladelphia, PA 19133",39.993999,-75.137940,0,,11,GM-10_GM-13_PM-34_PM-35_PM-39_PM-41_PM-42_PM-4...
3,PL-3,North Philadelphia,Fairhill,5th & Indiana Street Lot,"5th Street & Indiana Street Philadelphia, PA 1...",40.003162,-75.137864,0,,9,GM-10_GM-13_PM-39_PM-41_PM-43_PM-45_PM-50_PM-5...
4,PL-4,North Philadelphia,Strawberry Mansion,32nd & West Lehigh Avenue Lot,"3250 W. Lehigh AvenuePhiladelphia, PA 19132",39.997867,-75.184748,0,,5,GM-0_PM-26_PM-38_PM-44_PM-49_
5,PL-5,North Philadelphia,West Kensington,East Huntingdon Street Lot,"301 E. Huntingdon StreetPhiladelphia, PA 19125",39.989288,-75.127644,0,,11,GM-10_GM-12_GM-13_PM-34_PM-35_PM-41_PM-42_PM-4...
6,PL-6,North Philadelphia,West Poplar,6th & Marshall Street Lot,"982 N. 6th Street Philadelphia, PA 19123",39.970208,-75.146787,0,,17,GM-2_GM-9_GM-11_GM-14_GM-16_GM-18_PM-1_PM-4_PM...
7,PL-7,North Philadelphia,Logan,North Broad Street Lot,"4850 N. Broad StreetPhiladelphia, PA 19141",40.027449,-75.147563,0,,0,
8,PL-8,North Philadelphia,Logan,Fern Rock Rail Station Parking Lot,"901 Nedro AvenuePhiladelphia, PA 19141",40.042499,-75.135363,0,,0,
9,PL-9,North Philadelphia,Olney,Fairhill Street Lot,"5541 Fairhill StreetPhiladelphia, PA 19120",40.036382,-75.131405,0,,0,


In [340]:
df_parknrec=pd.read_csv('PPR_Program_Sites.csv')

In [341]:
df_parknrec.head(2)

,X,Y,OBJECTID,PARK_NAME,DPP_ASSET_ID,PROGRAM_TYPE,SITE_CLASS,BUILDING,GYM,LABEL_NUMBER,COMMENTS,DATA_SOURCE
0,-75.163634,40.053863,1,Awbury Park and Recreation Center,604,PPR_REC,B,Y,N,8.0,,Programs 9/2021
1,-75.007511,40.122843,2,Daniel Boyle Memorial Recreation Center,1843,PPR_REC,B,Y,N,13.0,,Programs 9/2021


In [342]:
df_parknrec=df_parknrec[df_parknrec['BUILDING']=='Y']

In [343]:
df_parknrec.head(2)

,X,Y,OBJECTID,PARK_NAME,DPP_ASSET_ID,PROGRAM_TYPE,SITE_CLASS,BUILDING,GYM,LABEL_NUMBER,COMMENTS,DATA_SOURCE
0,-75.163634,40.053863,1,Awbury Park and Recreation Center,604,PPR_REC,B,Y,N,8.0,,Programs 9/2021
1,-75.007511,40.122843,2,Daniel Boyle Memorial Recreation Center,1843,PPR_REC,B,Y,N,13.0,,Programs 9/2021


In [344]:
df_parknrec=df_parknrec.drop(['OBJECTID','DPP_ASSET_ID','PROGRAM_TYPE','SITE_CLASS','GYM','LABEL_NUMBER','COMMENTS','DATA_SOURCE'],axis=1)

In [345]:
df_parknrec.rename(columns={'X':'Long','Y':'Lat'},inplace=True)

In [346]:
df_parknrec['Id']=df_parknrec.apply(lambda a:'PNR-'+str(a.name),axis=1)

In [347]:
df_parknrec=df_parknrec[['Id','PARK_NAME','BUILDING','Lat','Long']]

In [348]:
df_parknrec.to_csv('parksnrec.csv',index=False)

In [349]:
df_parknrec.head(2)

,Id,PARK_NAME,BUILDING,Lat,Long
0,PNR-0,Awbury Park and Recreation Center,Y,40.053863,-75.163634
1,PNR-1,Daniel Boyle Memorial Recreation Center,Y,40.122843,-75.007511


In [350]:
df_parknrec['Wifi_Points']=0

In [351]:
df_parknrec['Wifi_Locs']=''

In [352]:
df_parknrec.head(2)

,Id,PARK_NAME,BUILDING,Lat,Long,Wifi_Points,Wifi_Locs
0,PNR-0,Awbury Park and Recreation Center,Y,40.053863,-75.163634,0,
1,PNR-1,Daniel Boyle Memorial Recreation Center,Y,40.122843,-75.007511,0,


In [353]:
# df_network.iloc[0,-1]+=('2'+'_')

In [354]:
df_lib_loc.head(2)

,Id,Name,Address,Lat,Long
0,WL-0,Andorra Library,"705 East Cathedral Road Philadelphia, PA 19128...",40.06489,-75.23608
1,WL-1,Blanche A. Nixon/Cobbs Creek Library,"5800 Cobbs Creek Parkway Philadelphia, PA 1914...",39.94588,-75.24119


In [355]:
df_lib_loc.iloc[0,0]

'WL-0'

In [356]:
par_lat,par_lon=df_parknrec.iloc[0,-4:-2].values
lib_lat,lib_lon=df_lib_loc.iloc[0,-2:].values
type(haversine((par_lat,par_lon),(lib_lat,lib_lon),unit=Unit.METERS))

float

In [357]:
for i in range(len(df_parknrec)):
    par_lat,par_lon=df_parknrec.iloc[i,-4:-2].values
    for j in range(len(df_lib_loc)):
        lib_lat,lib_lon=df_lib_loc.iloc[j,-2:].values
        dis=haversine((par_lat,par_lon),(lib_lat,lib_lon),unit=Unit.METERS)
        if dis<=100:
            df_parknrec.iloc[i,-2]+=1
            df_parknrec.iloc[i,-1]+=(df_lib_loc.iloc[j,0]+'_')
        

In [358]:
df_parknrec.head(2)

,Id,PARK_NAME,BUILDING,Lat,Long,Wifi_Points,Wifi_Locs
0,PNR-0,Awbury Park and Recreation Center,Y,40.053863,-75.163634,0,
1,PNR-1,Daniel Boyle Memorial Recreation Center,Y,40.122843,-75.007511,0,


In [359]:
df_parknrec[df_parknrec['Wifi_Points']!=0]

,Id,PARK_NAME,BUILDING,Lat,Long,Wifi_Points,Wifi_Locs
10,PNR-10,John C Anderson Cultural Center,Y,39.995218,-75.233243,1,WL-51_
17,PNR-17,A Giuseppe DiSilvestro Playground,Y,39.929140,-75.169962,1,WL-41_
37,PNR-37,Fishtown Recreation Center,Y,39.971852,-75.127804,1,WL-11_
91,PNR-91,Lawncrest Recreation Center,Y,40.046156,-75.101320,1,WL-24_
124,PNR-124,James L Wright Playground,Y,39.963768,-75.190867,1,WL-5_


In [360]:
df_meal.head(2)

,Id,Name,Address,Lat,Long
0,GM-0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.99039,-75.16421
1,GM-1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.94654,-75.16467


In [361]:
df_meal.iloc[0,-2:].values

array([39.99039, -75.16421], dtype=object)

In [362]:
df_parknrec['Meal_Points']=0

In [363]:
df_parknrec['Meal_Loc']=''

In [364]:
df_parknrec.head(2)

,Id,PARK_NAME,BUILDING,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PNR-0,Awbury Park and Recreation Center,Y,40.053863,-75.163634,0,,0,
1,PNR-1,Daniel Boyle Memorial Recreation Center,Y,40.122843,-75.007511,0,,0,


In [365]:
for i in range(len(df_parknrec)):
    par_lat,par_lon=df_parknrec.iloc[i,-6:-4].values
    for j in range(len(df_meal)):
        meal_lat,meal_lon=df_meal.iloc[j,-2:].values
        dis=haversine((par_lat,par_lon),(meal_lat,meal_lon),unit=Unit.KILOMETERS)
        if dis<=2:
            df_parknrec.iloc[i,-2]+=1
            df_parknrec.iloc[i,-1]+=(df_meal.iloc[j,0]+'_')

In [366]:
df_network.head(2)

,Id,Region,Area,Name,Address,Lat,Long,Wifi_Points,Wifi_Locs,Meal_Points,Meal_Loc
0,PL-0,North Philadelphia,Brewerytown,22nd & Turner Street Lot,"2200 Turner StreetPhiladelphia, PA 19121",39.979229,-75.171289,0,,13,GM-0_GM-4_PM-10_PM-16_PM-18_PM-25_PM-26_PM-32_...
1,PL-1,North Philadelphia,Fairmount,19th & Callowhill,"900 Callowhill StreetPhiladelphia, PA 19130",39.958586,-75.153734,0,,25,GM-1_GM-2_GM-6_GM-7_GM-8_GM-9_GM-11_GM-14_GM-1...


In [374]:
sum(df_parknrec['Meal_Points']!=0)

96

In [ ]:
df_network[df_parknrec['Meal_Points']!=0]

In [376]:
df_meal

,Id,Name,Address,Lat,Long
0,GM-0,Bethel Presbyterian General meal site,"1900 W York St Philadelphia, PA 19132",39.990390,-75.164210
1,GM-1,Broad Street Ministry General meal site,"315 S Broad Street Philadelphia, PA 19107",39.946540,-75.164670
2,GM-2,Chosen 300 General meal site,"1116 Spring Garden St. Philadelphia, PA 19123",39.961570,-75.156300
3,GM-3,Chosen 300 West General meal site,"3959 Lancaster Ave Philadelphia, PA 19104",39.963050,-75.202010
4,GM-4,Church of the Advocate General meal site,"1801 Diamond St Philadelphia, PA 19121",39.985690,-75.163370
...,...,...,...,...,...
76,PM-53,Holy Temple Lord and Savior,5116 Market Street Philadelphia PA 19139 Unite...,39.959810,-75.224240
77,PM-54,Church of the Holy Apostles,260 South 51st Street Philadelphia PA 19139 Un...,39.954680,-75.224080
78,PM-55,Pente Church of God,3027 North 5th Street Philadelphia PA 19133 Un...,39.997410,-75.138920
79,PM-56,Cathedral Kitchen,1514 Federal St Camden NJ 8105 United States,39.944074,-75.102257


In [372]:
df_meal.shape

(81, 5)

In [373]:
df_network.to_csv('network_parkinglot.csv',index=False)

In [377]:
df_parknrec.to_csv('network_parksnrec.csv',index=False)

In [378]:
# Network Points for Parking Lot

In [382]:
sum(df_network['Meal_Points']!=0)

24

In [384]:
# Areas to be observed for Network Points

In [385]:
sum(df_network['Meal_Points']==0)

13

In [386]:
# Network Points for ParksnRecs

In [388]:
sum(df_parknrec['Meal_Points']!=0)

96

In [390]:
# Areas to be observed for Network Points

In [391]:
sum(df_parknrec['Meal_Points']==0)

86